In [0]:
import heapq
import pandas as pd
import numpy as np
import nltk
import time
import math
import random
import bisect
import collections
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

data = pd.read_csv('./results.csv', sep=',')
queries = ["lula", "bolsonaro", "brasil", "economia", "ditadura"]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Abaixo iremos criar o ranking a partir dos dados coletados ainda na primeira atividade do curso.
Essa atividade já foi feita na atividade anterior e se trata apenas da criação de um índice invertido.

In [0]:
textData = pd.DataFrame(data['text'], columns=['text'])
textData['tokenizedText'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)

stopwords = nltk.corpus.stopwords.words('portuguese')

index = {}
document = 0

for wordList in textData.tokenizedText:
  document += 1
  for word in wordList:
    if word not in stopwords and len(word) >= 3:      
      if word not in index.keys():
        index[word] = []
      index[word].append(document)
      
for elem in index.items():
  d = dict(collections.Counter(elem[1]))
  index[elem[0]] = list(d.items())

#Questão 1

Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? 


Para realizarmos os cáculos dessas métricas, iremos precisar definir algumas funções que recuperaram os seguintes valores

*   A quantidade de documentos em que uma palavra aparece
*   A quantidade de documentos em que duas palavras aparecem simultaneamente



In [0]:
def doc_word_occurence(word, ranking):
  return len(ranking[word])

def doc_two_words_occurence(word1, word2, ranking):
  documents_a = [doc for doc,score in ranking[word1]]
  documents_b = [doc for doc,score in ranking[word2]]
  occurences = 0
  
  for document in documents_a:
    if document in documents_b:
      occurences += 1
      
  return occurences

Abaixo iremos criar as funções necessárias para a realização do cáculo de cada uma das métricas.




In [0]:
def mim(word1, word2, ranking):
  word1_occurences = doc_word_occurence(word1, ranking)
  word2_occurences = doc_word_occurence(word2, ranking)
  both_words_occurences = doc_two_words_occurence(word1, word2, ranking)
  quo = (word1_occurences * word2_occurences)
  
  if (quo != 0):
    mim = both_words_occurences / quo
  else:
    mim = 0
  
  return (mim,word2)


def emim(word1, word2, ranking, n):
  word1_occurences = doc_word_occurence(word1, ranking)
  word2_occurences = doc_word_occurence(word2, ranking)
  both_words_occurences = doc_two_words_occurence(word1, word2, ranking)
  quo = (word1_occurences * word2_occurences)
  
  emim = 0
  
  if (quo != 0):
    exp = n * (both_words_occurences / quo)
    if (exp != 0):
      emim = both_words_occurences * np.log10(exp)
 
  return (emim, word2)

  
def x2(word1, word2, ranking, n):
  word1_occurences = doc_word_occurence(word1, ranking)
  word2_occurences = doc_word_occurence(word2, ranking)
  both_words_occurences = doc_two_words_occurence(word1, word2, ranking)
  quo = (word1_occurences * word2_occurences)
  
  if (quo != 0):
    x = (both_words_occurences - (1/n) * word1_occurences * word2_occurences) ** 2 / quo
  else:
    x = 0

  return (x,word2)
  
def dice(word1, word2, ranking):
  word1_occurences = doc_word_occurence(word1, ranking)
  word2_occurences = doc_word_occurence(word2, ranking)
  both_words_occurences = doc_two_words_occurence(word1, word2, ranking)
  
  dice = both_words_occurences / (word1_occurences + word2_occurences)
  
  return (dice,word2)

In [0]:
#função para criar a tabela pedida no exercício.

def create_table(query, ranking, n):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, mim(query, word, ranking))
      bisect.insort(EMIM,emim(query, word, ranking, n))
      bisect.insort(X,x2(query, word, ranking, n))
      bisect.insort(Dice,dice(query, word, ranking))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM[:10]]
  top_10_table['EMIM'] = [word[1] for word in EMIM[:10]]
  top_10_table['X²'] = [word[1] for word in X[:10]]
  top_10_table['DICE'] = [word[1] for word in Dice[:10]]
  
  top_10_table.index += 1
  
  return top_10_table

In [0]:
n = data.text.count() #Total de documentos na coleção.

table_lula = create_table(queries[0], index, n)
table_bolsonaro = create_table(queries[1], index, n)
table_brasil = create_table(queries[2], index, n)
table_economia = create_table(queries[3], index, n)
table_ditadura = create_table(queries[4], index, n)

Abaixo iremos mostrar as top-10 palavras associadas a cada uma das palavras do nosso query de acordo com cada uma das métricas.

In [0]:
table_lula

,MIM,EMIM,X²,DICE
1,—em,ex-presidente,inácio,inácio
2,–mais,inácio,ex-presidente,ex-presidente
3,–15,bolsonaro,psicanálise,silva
4,ótimo/excelente,prisão,pareceram,michel
5,ácidas,silva,minimiza,entregar
6,xavier.o,temer,interrogado,temer
7,xavier,michel,governabilidade,prisão
8,vomitando,reforma,ex-juiz,recado
9,voltas,preso,eleitorado,psol
10,vizinho.na,entregar,desidratação,negativa


In [0]:
table_bolsonaro


,MIM,EMIM,X²,DICE
1,👍🇧🇷🌎,presidente,previdência,presidente
2,✊🏻✊🏼✊🏼✊🏽✊🏾✊🏿📷,brasil,reforma,brasil
3,—sim,brasileiro,brasileiro,brasileiro
4,—ou,ministro,ministro,governo
5,—os,governo,presidente,ministro
6,—local,reforma,psl,político
7,—aquelas,político,deputado,brasileira
8,–yuriy,previdência,jair,afirmou
9,–parte,brasileira,guedes,reforma
10,–outro,deputado,brasil,paulo


In [0]:
table_brasil

,MIM,EMIM,X²,DICE
1,👍🇧🇷🌎,bolsonaro,brasileiros,bolsonaro
2,✊🏻✊🏼✊🏼✊🏽✊🏾✊🏿📷,brasileiros,bolsonaro,sobre
3,—ver,brasileira,brasileira,ainda
4,—uma,sociais,brasileiro,governo
5,—tios,paulo,paulo,presidente
6,—sim,brasileiro,sociais,ser
7,—plasmando,governo,etc,sociais
8,—para,presidente,brasileiras,anos
9,—ou,ainda,direita,país
10,—os,sobre,regime,ter


In [0]:
table_economia

,MIM,EMIM,X²,DICE
1,👍🇧🇷🌎,reforma,guedes,reforma
2,—sim,previdência,previdência,previdência
3,—ou,mercado,reforma,mercado
4,—desde,guedes,mercados,governo
5,—aquelas,governo,reformas,econômica
6,–yuriy,econômica,mercado,ministro
7,–parte,ministro,aposentadorias,política
8,–outro,reformas,incerteza,menos
9,–mais,política,aposentadoria,guedes
10,–15,menos,maia,bolsonaro


In [0]:
table_ditadura

,MIM,EMIM,X²,DICE
1,✊🏻✊🏼✊🏼✊🏽✊🏾✊🏿📷,militar,militar,militar
2,—tios,militares,golpe,militares
3,—os,golpe,militares,golpe
4,—inclusive,regime,quartéis,regime
5,—de,quartéis,1964,quartéis
6,ósseos,1964,regime,1964
7,ópio,exército,torturador,exército
8,ético,bolsonaro,repressão,naquele
9,épico,vítimas,militantes,crimes
10,ângulos,naquele,chumbo,vítimas


Analisando os resultados de cada métrica, podemos identificar que das quatro, apenas o **MIM** não apresenta resultados muito satisfatórios, considerando, as palvras que mais se encaixariam no contexto das palavras das queries.

Embora as três outras métricas tenham resultados muito parecidos, elegemos a **DICE** como a melhor delas pela simplicidade de implementação.

#Questão 2

De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

Utilizaremos aqui o algoritmo de consulta termo-por-vez já implementado na atividade anterior e o algoritmo que pega os top-k documentos. 

In [0]:
def termAtATimeRetrieval(query, inverted_index, k):
    a = {}
    inverted_lists = []
    r = []
    for word in query.split(" "):
        if word in inverted_index.keys():
            inverted_lists.append(inverted_index[word])
    for inverted_list in inverted_lists:
        for post in inverted_list:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))

    return heapq.nlargest(k, r)


def top_documents(query, k):
  
  top_docs = [doc for score,doc in termAtATimeRetrieval(query, index, k)]
  top_docs_ranking = {} 
  
  for word in index:
    top_docs_ranking[word] = []
    for doc_score in index[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

##Top-3 Documentos

Abaixo faremos a expansão com os top 3 documentos de acordo com os queries selecionados previamente.

In [0]:
k = 3
ranking_docs_lula = top_documents(queries[0], k)
ranking_docs_bolsonaro = top_documents(queries[1], k)
ranking_docs_brasil = top_documents(queries[2], k)
ranking_docs_economia = top_documents(queries[3], k)
ranking_docs_ditadura = top_documents(queries[4], k)

In [0]:
table_top_lula = create_table(queries[0], ranking_docs_lula, k)
table_top_bolsonaro = create_table(queries[1], ranking_docs_bolsonaro, k)
table_top_brasil = create_table(queries[2], ranking_docs_brasil, k)
table_top_economia = create_table(queries[3], ranking_docs_economia, k)
table_top_ditadura = create_table(queries[4], ranking_docs_ditadura, k)

Abaixo iremos mostrar as tabelas referentes a cada uma das query para realizar a comparação antes e após a expansão

In [0]:
table_dice_lula = pd.DataFrame()
table_dice_lula['DICE - before'] = table_lula['DICE']
table_dice_lula['DICE - after'] = table_top_lula['DICE']
table_dice_lula

,DICE - before,DICE - after
1,inácio,ter
2,ex-presidente,ser
3,silva,político
4,michel,pode
5,entregar,outro
6,temer,contra
7,prisão,brasil
8,recado,bolsonaro
9,psol,ódio
10,negativa,época


In [0]:
table_dice_bolsonaro = pd.DataFrame()
table_dice_bolsonaro['DICE - before'] = table_bolsonaro['DICE']
table_dice_bolsonaro['DICE - after'] = table_top_bolsonaro['DICE']
table_dice_bolsonaro

,DICE - before,DICE - after
1,presidente,vez
2,brasil,três
3,brasileiro,trata
4,governo,trabalho
5,ministro,todas
6,político,sobre
7,brasileira,sob
8,afirmou,site
9,reforma,sendo
10,paulo,república


In [0]:
table_dice_brasil = pd.DataFrame()
table_dice_brasil['DICE - before'] = table_brasil['DICE']
table_dice_brasil['DICE - after'] = table_top_brasil['DICE']
table_dice_brasil

,DICE - before,DICE - after
1,bolsonaro,vão
2,sobre,voto
3,ainda,violência
4,governo,vida
5,presidente,vezes
6,ser,vez
7,sociais,vergonha
8,anos,verdade
9,país,vai
10,ter,têm


In [0]:
table_dice_economia = pd.DataFrame()
table_dice_economia['DICE - before'] = table_economia['DICE']
table_dice_economia['DICE - after'] = table_top_economia['DICE']
table_dice_economia

,DICE - before,DICE - after
1,reforma,sobre
2,previdência,setor
3,mercado,ser
4,governo,segundo
5,econômica,resultado
6,ministro,relação
7,política,quase
8,menos,principalmente
9,guedes,pouco
10,bolsonaro,pesquisa


In [0]:
table_dice_ditadura = pd.DataFrame()
table_dice_ditadura['DICE - before'] = table_ditadura['DICE']
table_dice_ditadura['DICE - after'] = table_top_ditadura['DICE']
table_dice_ditadura

,DICE - before,DICE - after
1,militar,época
2,militares,vai
3,golpe,todos
4,regime,ter
5,quartéis,tempo
6,1964,tanto
7,exército,ser
8,naquele,regime
9,crimes,preso
10,vítimas,presidente


##Top-5 Documentos

Abaixo faremos a expansão com os top 5 documentos de acordo com os queries selecionados previamente. O processo é o mesmo que o de top-3 documentos, a única diferença será o K utilizado.

In [0]:
k = 5
ranking_docs_lula = top_documents(queries[0], k)
ranking_docs_bolsonaro = top_documents(queries[1], k)
ranking_docs_brasil = top_documents(queries[2], k)
ranking_docs_economia = top_documents(queries[3], k)
ranking_docs_ditadura = top_documents(queries[4], k)

table_top_lula = create_table(queries[0], ranking_docs_lula, k)
table_top_bolsonaro = create_table(queries[1], ranking_docs_bolsonaro, k)
table_top_brasil = create_table(queries[2], ranking_docs_brasil, k)
table_top_economia = create_table(queries[3], ranking_docs_economia, k)
table_top_ditadura = create_table(queries[4], ranking_docs_ditadura, k)

In [0]:
table_dice_lula = pd.DataFrame()
table_dice_lula['DICE - before'] = table_lula['DICE']
table_dice_lula['DICE - after'] = table_top_lula['DICE']
table_dice_lula

,DICE - before,DICE - after
1,inácio,ser
2,ex-presidente,todos
3,silva,ter
4,michel,sobre
5,entregar,prisão
6,temer,preso
7,prisão,presidente
8,recado,político
9,psol,pode
10,negativa,outro


In [0]:
table_dice_bolsonaro = pd.DataFrame()
table_dice_bolsonaro['DICE - before'] = table_bolsonaro['DICE']
table_dice_bolsonaro['DICE - after'] = table_top_bolsonaro['DICE']
table_dice_bolsonaro

,DICE - before,DICE - after
1,presidente,três
2,brasil,sobre
3,brasileiro,república
4,governo,presidente
5,ministro,parte
6,político,oficial
7,brasileira,ministério
8,afirmou,fazer
9,reforma,dois
10,paulo,dia


In [0]:
table_dice_brasil = pd.DataFrame()
table_dice_brasil['DICE - before'] = table_brasil['DICE']
table_dice_brasil['DICE - after'] = table_top_brasil['DICE']
table_dice_brasil

,DICE - before,DICE - after
1,bolsonaro,vida
2,sobre,vai
3,ainda,sobre
4,governo,país
5,presidente,parte
6,ser,mundo
7,sociais,maior
8,anos,governo
9,país,fazer
10,ter,faz


In [0]:
table_dice_economia = pd.DataFrame()
table_dice_economia['DICE - before'] = table_economia['DICE']
table_dice_economia['DICE - after'] = table_top_economia['DICE']
table_dice_economia

,DICE - before,DICE - after
1,reforma,sobre
2,previdência,ser
3,mercado,relação
4,governo,pouco
5,econômica,mercado
6,ministro,dinheiro
7,política,crise
8,menos,agora
9,guedes,vez
10,bolsonaro,tudo


In [0]:
table_dice_ditadura = pd.DataFrame()
table_dice_ditadura['DICE - before'] = table_ditadura['DICE']
table_dice_ditadura['DICE - after'] = table_top_ditadura['DICE']
table_dice_ditadura

,DICE - before,DICE - after
1,militar,todos
2,militares,presidente
3,golpe,pessoas
4,regime,militar
5,quartéis,diz
6,1964,cada
7,exército,bolsonaro
8,naquele,anos
9,crimes,época
10,vítimas,ter


##Top-10 Documentos

In [0]:
k = 10
ranking_docs_lula = top_documents(queries[0], k)
ranking_docs_bolsonaro = top_documents(queries[1], k)
ranking_docs_brasil = top_documents(queries[2], k)
ranking_docs_economia = top_documents(queries[3], k)
ranking_docs_ditadura = top_documents(queries[4], k)

table_top_lula = create_table(queries[0], ranking_docs_lula, k)
table_top_bolsonaro = create_table(queries[1], ranking_docs_bolsonaro, k)
table_top_brasil = create_table(queries[2], ranking_docs_brasil, k)
table_top_economia = create_table(queries[3], ranking_docs_economia, k)
table_top_ditadura = create_table(queries[4], ranking_docs_ditadura, k)

In [0]:
table_dice_lula = pd.DataFrame()
table_dice_lula['DICE - before'] = table_lula['DICE']
table_dice_lula['DICE - after'] = table_top_lula['DICE']
table_dice_lula

,DICE - before,DICE - after
1,inácio,ser
2,ex-presidente,bolsonaro
3,silva,presidente
4,michel,pode
5,entregar,ainda
6,temer,todos
7,prisão,ter
8,recado,político
9,psol,política
10,negativa,outros


In [0]:
table_dice_bolsonaro = pd.DataFrame()
table_dice_bolsonaro['DICE - before'] = table_bolsonaro['DICE']
table_dice_bolsonaro['DICE - after'] = table_top_bolsonaro['DICE']
table_dice_bolsonaro

,DICE - before,DICE - after
1,presidente,presidente
2,brasil,fazer
3,brasileiro,sobre
4,governo,parte
5,ministro,ainda
6,político,ser
7,brasileira,político
8,afirmou,outros
9,reforma,ministério
10,paulo,governo


In [0]:
table_dice_brasil = pd.DataFrame()
table_dice_brasil['DICE - before'] = table_brasil['DICE']
table_dice_brasil['DICE - after'] = table_top_brasil['DICE']
table_dice_brasil

,DICE - before,DICE - after
1,bolsonaro,país
2,sobre,parte
3,ainda,governo
4,governo,vida
5,presidente,sobre
6,ser,ser
7,sociais,brasileiro
8,anos,brasileira
9,país,anos
10,ter,ainda


In [0]:
table_dice_economia = pd.DataFrame()
table_dice_economia['DICE - before'] = table_economia['DICE']
table_dice_economia['DICE - after'] = table_top_economia['DICE']
table_dice_economia

,DICE - before,DICE - after
1,reforma,sobre
2,previdência,ser
3,mercado,todos
4,governo,ter
5,econômica,outros
6,ministro,ainda
7,política,relação
8,menos,pouco
9,guedes,porque
10,bolsonaro,pode


In [0]:
table_dice_ditadura = pd.DataFrame()
table_dice_ditadura['DICE - before'] = table_ditadura['DICE']
table_dice_ditadura['DICE - after'] = table_top_ditadura['DICE']
table_dice_ditadura

,DICE - before,DICE - after
1,militar,todos
2,militares,militar
3,golpe,anos
4,regime,presidente
5,quartéis,pessoas
6,1964,militares
7,exército,história
8,naquele,ainda
9,crimes,sobre
10,vítimas,regime


##Conclusão

Ao analisarmos as tabelas acima, podemos concluir que 

*   Top-3 documentos
          Perdemos em eficiência, as palavras antes da expansão fazem muito mais sentido no contexto do que as após a expansão.
*   Top-5 documentos
          Podemos concluir uma leve melhora na eficiência comparado à expansão anterior, no entanto, os resultados anteriores a expansão ainda são melhores.       
*   Top-10 documentos
          Essa é a expansão que nos trás os melhores resultados, pois trás contextos diferentes para a palavra buscada mas que fazem sentido, levando o contexto em consideração.

